# 0.6 Train SVM model

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import os
import string

In [10]:
cd /Users/anacriel/plant_seed_classification/src

/Users/anacriel/plant_seed_classification/src


In [13]:
dataset = pd.read_csv("data/advanced_features.csv")

In [20]:
dataset.drop(columns='inverse_difference_moments', inplace=True)

In [21]:
X = dataset.iloc[:, 2:]
X.head(5)

,area,perimeter,physiological_length,physiological_width,aspect_ratio,rectangularity,circularity,mean_r,mean_g,mean_b,stddev_r,stddev_g,stddev_b,contrast,correlation,entropy
0,4731.5,289.036578,75,88,0.852273,1.394906,17.656587,10.813889,12.858433,6.945078,21.764148,25.679687,14.463592,387.330372,0.970058,2.842585
1,8901.5,426.717817,125,121,1.033058,1.699152,20.455889,13.619089,16.084000,7.870911,25.215709,29.673654,14.943225,354.426094,0.973122,2.902780
2,4942.5,341.462983,84,91,0.923077,1.546586,23.590687,10.316633,12.143667,6.440978,23.062125,26.891960,14.631709,300.822603,0.971422,2.338985
3,9126.5,387.119838,98,132,0.742424,1.417411,16.420508,12.527533,14.733822,7.119733,24.378071,28.472611,14.053283,233.826945,0.981433,2.536083
4,310.5,77.355339,21,27,0.777778,1.826087,19.271653,15.972422,18.640944,10.102156,25.636130,29.667084,16.636622,350.008474,0.978098,3.557184


In [25]:
y = dataset.iloc[:,1]
y.head(5)

0    9
1    9
2    9
3    9
4    9
Name: label, dtype: int64

## Cross validational split

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 142)

In [28]:
X_train.head(5)

,area,perimeter,physiological_length,physiological_width,aspect_ratio,rectangularity,circularity,mean_r,mean_g,mean_b,stddev_r,stddev_g,stddev_b,contrast,correlation,entropy
3465,11.5,23.071068,11,4,2.750000,3.826087,46.284710,1.846633,2.156744,0.842411,11.257446,13.024320,5.410962,182.013162,0.899611,0.475508
467,241.0,91.112698,34,18,1.888889,2.539419,34.446156,15.406356,18.065978,6.113956,31.565384,36.923046,13.684025,316.324080,0.966831,2.646134
963,18092.0,1403.935197,164,289,0.567474,2.619721,108.945061,12.371022,15.279767,4.537900,24.633182,30.212064,10.390288,370.835412,0.969445,2.764188
1471,247.0,136.166520,48,42,1.142857,8.161943,75.066078,4.635300,4.776444,2.306133,19.079526,19.467070,9.915341,464.086588,0.868308,1.012637
4116,1840.0,250.994947,109,29,3.758621,1.717935,34.238295,22.046044,26.012422,8.390289,35.252506,40.956261,15.419196,477.671442,0.963758,3.942430


In [29]:
y_train[0:5]

3465    11
467      0
963      8
1471    10
4116     4
Name: label, dtype: int64

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

/Users/anacriel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/anacriel/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/anacriel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
X_train[0:2]

array([[-0.84505668, -0.9627431 , -1.00512438, -1.06132362,  1.33896381,
         0.21921927,  0.02946126, -1.52512799, -1.47915781, -1.40722134,
        -1.9886983 , -1.84336353, -1.88829048, -0.67107257, -1.13299426,
        -1.52906104],
       [-0.82214663, -0.81210094, -0.78314566, -0.92561622,  0.55134859,
        -0.11437675, -0.17960225,  0.33079501,  0.35327988,  0.17912413,
         0.7622453 ,  0.83124596,  0.41087199,  0.23406286,  0.25918872,
         0.24303969]])

In [33]:
y_train[0:2]

3465    11
467      0
Name: label, dtype: int64

## Applying SVM classifier model

In [34]:
from sklearn import svm

In [35]:
clf = svm.SVC()
clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [36]:
y_pred = clf.predict(X_test)

In [37]:
from sklearn import metrics

In [38]:
metrics.accuracy_score(y_test, y_pred)

0.735252808988764

In [39]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.69      0.68       128
           1       0.70      0.72      0.71       138
           2       0.64      0.74      0.68       148
           3       0.76      0.82      0.79       115
           4       0.71      0.90      0.79       144
           5       0.83      0.55      0.66        62
           6       0.69      0.54      0.61        67
           7       0.64      0.42      0.51        60
           8       0.93      0.91      0.92       190
           9       0.81      0.79      0.80        89
          10       0.70      0.87      0.78       212
          11       0.50      0.07      0.12        71

   micro avg       0.74      0.74      0.74      1424
   macro avg       0.72      0.67      0.67      1424
weighted avg       0.73      0.74      0.72      1424



## Performing parameter tuning of the model

In [40]:
from sklearn.model_selection import GridSearchCV

In [41]:
parameters = [{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
               'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
             ]

In [42]:
svm_clf = GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=5)
svm_clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
svm_clf.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [44]:
means = svm_clf.cv_results_['mean_test_score']
stds = svm_clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, svm_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.252 (+/-0.006) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.461 (+/-0.017) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.616 (+/-0.018) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.724 (+/-0.012) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.732 (+/-0.013) for {'C': 1, 'gamma': 0.2, 'kernel': 'rbf'}
0.726 (+/-0.024) for {'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}
0.460 (+/-0.016) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.614 (+/-0.024) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.731 (+/-0.020) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.769 (+/-0.021) for {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.765 (+/-0.019) for {'C': 10, 'gamma': 0.2, 'kernel': 'rbf'}
0.738 (+/-0.029) for {'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}
0.609 (+/-0.021) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.719 (+/-0.018) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.781 (+/-0.014) for {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
0.779 (+/-0.017) for {'C': 100, 'gamma': 0.1, 'kernel':

In [45]:
y_pred_svm = svm_clf.predict(X_test)

In [46]:
metrics.accuracy_score(y_test, y_pred_svm)

0.8209269662921348

In [47]:
print(metrics.classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.78      0.82      0.80       128
           1       0.76      0.81      0.78       138
           2       0.76      0.86      0.81       148
           3       0.91      0.83      0.87       115
           4       0.82      0.88      0.85       144
           5       0.88      0.81      0.84        62
           6       0.75      0.69      0.72        67
           7       0.80      0.85      0.82        60
           8       0.97      0.92      0.94       190
           9       0.89      0.83      0.86        89
          10       0.83      0.83      0.83       212
          11       0.54      0.45      0.49        71

   micro avg       0.82      0.82      0.82      1424
   macro avg       0.81      0.80      0.80      1424
weighted avg       0.82      0.82      0.82      1424



## Testing random image